# Fetch Tile Index Files
This notebook downloads the tile index layers, in geopackage format, to the local Box drive. All 13 project geopackages are downloaded, the tile feature classes are merged into a single feature class named `USGS_Tiles` in the `USGS3DEP_Indices.gdb`

### Manually downloading the geopackages:
1. Navigate to <https://prd-tnm.s3.amazonaws.com/index.html?prefix=StagedProducts/Elevation/metadata/NC_HurricaneFlorence_2020_D20>
1. Click on one of the project folders, eg `NC_HurFlo_NOAA_12_2020`
1. Navigate to `spatial_metadata`\\`USGS`
1. Click on the `lpc_index.gpkg` to download the file

### Download links
* `https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/metadata/NC_HurricaneFlorence_2020_D20/<project name>/spatial_metadata/USGS/lpc_index.gpkg`<br>
replacing `<project name>` with the project name.

**CONDA ENVIRONMENT**: `arcgispro-py3`

In [ ]:
#Packages
from pathlib import Path
from urllib import request
import numpy as np
import pandas as pd
import arcpy

In [ ]:
#Set the output folder on Box
save_fldr = Path.home() / 'Box' / 'PROJECT 05-21-2025□ USGS3DEP' / 'Tile_Indices'
if not save_fldr.exists(): 
    print("Folder not found. Is Box drive running?")

In [ ]:
#Create a geodatabase in the Box folder
out_gdb = arcpy.management.CreateFileGDB(
    out_folder_path = str(save_fldr),
    out_name = "USGS3DEP_Indices"
)[0]

In [ ]:
#Data paths
project_url='https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/metadata/NC_HurricaneFlorence_2020_D20/{}/spatial_metadata/USGS/lpc_index.gpkg'

#Sub folders
sub_folders = [
    'NC_HurFlo_NOAA_12_2020',
    'NC_HurFlo_NOAA_13_2020',
    'NC_HurFlo_NOAA_14_2020',
    'NC_HurFlo_NOAA_15_2020',
    'NC_HurricaneFlorence_10_2020',
    'NC_HurricaneFlorence_11_2020',
    'NC_HurricaneFlorence_1_2020',
    'NC_HurricaneFlorence_2_2020',
    'NC_HurricaneFlorence_3_2020',
    'NC_HurricaneFlorence_4_2020',
    'NC_HurricaneFlorence_5_2020',
    'NC_HurricaneFlorence_6_2020',
    'NC_HurricaneFlorence_7_2020',
    'NC_HurricaneFlorence_8_2020',
    'NC_HurricaneFlorence_9_2020'
]


In [ ]:
#Iterate through the projects and fetch the geopackages, renaming the output with the project name
for sub_folder in sub_folders:
    #Fetch the geopackage
    out_file = save_fldr / f'{sub_folder}.gpkg'

    #Skip if already downloaded
    if out_file.exists(): 
        print(f'{out_file.name} already fetched.')
        continue
    #Fetch the laz file 
    else:
        gpkg_url = project_url.format(sub_folder)
        print(f'Fetching {out_file.name}')
        request.urlretrieve(gpkg_url, str(out_file))
                    

In [ ]:
#Extract the tile feature classes to a single geodatabase
for sub_folder in sub_folders:
    out_file = save_fldr / f'{sub_folder}.gpkg'
    arcpy.env.workspace = str(out_file)
    fcs = arcpy.ListFeatureClasses()
    for fc in fcs: 
        print(f'Saving {sub_folder} to {out_gdb}')
        #arcpy.conversion.ExportFeatures(
        #    in_features=fc,
        #    out_features = out_gdb + "/" + sub_folder
        #)

In [ ]:
#Create a list of all the feature classes in the respective geopackage
tile_fcs = []
for sub_folder in sub_folders:
    out_file = save_fldr / f'{sub_folder}.gpkg'
    arcpy.env.workspace = str(out_file)
    fc = arcpy.ListFeatureClasses()[0]
    tile_fcs.append(str(Path(out_file,fc)))   
#Merge them together into a new feature class
arcpy.management.Merge(
    inputs = tile_fcs,
    output = out_gdb+'\\USGS_Tiles'
)